In [13]:
# установка последнего lightautoml
# !pip install -q git+https://github.com/sb-ai-lab/LightAutoML.git

# Обучение автомл модели

In [14]:
import os
if 'drive' not in os.listdir():
    from google.colab import drive
    drive.mount('/content/drive')

path_to_folder = '/content/drive/MyDrive/psb_hack'

In [15]:
# standard libraries
import sys
import os
import json
import re
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
# ds libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
import torch
# lama
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task
# save/load model
import joblib
import gc
import yaml
# свои функции по оценке метрик
path_to_model_utils = '/content/drive/MyDrive/psb_hack'
sys.path.append(os.path.join(path_to_model_utils,'model_utils'))
from model_estimation import *

## Создание конфига

In [16]:
full_train_flag = False
features_pack = 'features_pack_v4'
features_selector = 'automl' # cb, automl, None
CONFIG_NAME = 'psb_hack_lama'
VERSION_MODEL = 543


algos = ['cb','cb_tuned','lgb','lgb_tuned','linear_l2','xgb', 'xgb_tuned'] # 5+
# algos = ['cb','lgb','linear_l2','xgb',] #1+
# algos = ['cb','lgb_tuned','linear_l2', 'xgb_tuned']# 2+
# algos = ['cb','cb_tuned','lgb','lgb_tuned','linear_l2','xgb', 'xgb_tuned']# 3+
# algos = ['dense', 'denselight', 'resnet', 'snn', 'autoint','linear_l2'] # 4+

config_model = dict(
                    n_threads = 4,
                    n_folds = 5,
                    random_state = 42,
                    task = dict(name='binary',metric='auc', loss='logloss',greater_is_better=True),
                    general_params = {'use_algos': [algos],
                                      'weighted_blender_max_nonzero_coef' : 0.05,},
                    advanced_roles = False,
                    roles = {'target': 'target',
                             'category': ['Способ оплаты', 'Источник', 'Категория номера'],
                             'drop' : ['Дата бронирования','Дата отмены', 'Заезд', 'Выезд',
                                       'Категория номера', 'mon','Статус брони','№ брони']},

                    model_name = f'{CONFIG_NAME}_v{VERSION_MODEL}',
                    features_pack = features_pack,
                    features_selector = features_selector,
                    full_train_flag = full_train_flag,
    )


print(f'{CONFIG_NAME}_v{VERSION_MODEL}.yaml')


with open(os.path.join(path_to_folder, 'configs', f'{CONFIG_NAME}_v{VERSION_MODEL}.yaml'),
          'w',encoding='utf-8') as config:
    yaml.dump(config_model,config)

# название файлов сохранения
features_file = os.path.join(path_to_folder,'data',features_pack + '_' + f'{features_selector}_features.xlsx')
model_file =  os.path.join(path_to_folder,'models',config_model['model_name'] + '.pkl')

psb_hack_lama_v543.yaml
psb_hack_lama_v543.yaml


## Загрузка данных

In [17]:
# чтение данных
dates = ['Дата бронирования','Дата отмены', 'Заезд', 'Выезд']
read_cols = pd.read_excel(features_file)['Feature'] if features_selector else None
train_full = pd.read_csv(os.path.join(path_to_folder,'data',f'full_train_{features_pack}.csv'),parse_dates=dates,
                         usecols=read_cols)
train = pd.read_csv(os.path.join(path_to_folder,'data',f'train_{features_pack}.csv'),parse_dates=dates,
                    usecols=read_cols)
oot_val = pd.read_csv(os.path.join(path_to_folder,'data',f'oot_val_{features_pack}.csv'),parse_dates=dates,
                      usecols=read_cols)
oof_val = pd.read_csv(os.path.join(path_to_folder,'data',f'oof_val_{features_pack}.csv'),parse_dates=dates,
                      usecols=read_cols)

gc.collect()

96

410

# Модель

**Подготовка модели**

In [18]:
task = Task(name=config_model['task']['name'],
                metric=config_model['task']['metric'],
                loss=config_model['task']['loss'],
                greater_is_better=config_model['task']['greater_is_better'])
reader_params = {'n_jobs' : config_model['n_threads'],
                 'cv': config_model['n_folds'],
                 'random_state' : config_model['random_state'],
                 'advanced_roles' : config_model['advanced_roles']
                 }

model = TabularAutoML(
        task = task,
        timeout= 60*60*10,
        gpu_ids= 0 if torch.cuda.is_available() else None,
        cpu_limit=-1,
        general_params=config_model['general_params'],
        reader_params=reader_params,
        tuning_params=config_model.get('tuning_params', {'max_tuning_iter': 5}),
        selection_params={'mode':0},
        nn_params = {
        "n_epochs": 5,
        "bs": 128,
        "num_workers": 0,
        "path_to_save": None,
        "freeze_defaults": True,
        "cont_embedder": "plr",
        "hidden_size": 32,
        'device': torch.device('cuda') if torch.cuda.is_available() else 'cpu',
    },
    nn_pipeline_params = {"use_qnt": True, "use_te": False},
)
cv = StratifiedKFold(n_splits=config_model['n_folds'],shuffle=True, random_state=config_model['random_state'])


cv_iters = list(cv.split(train_full if full_train_flag else train,
                         train_full[config_model['roles']['target']] if full_train_flag else train[config_model['roles']['target']]))

**Обучение модели**

In [19]:
%%time
train_pred = model.fit_predict(train_full if full_train_flag else train,
                               roles=config_model['roles'],
                               verbose=1,
                               cv_iter=cv_iters)

[13:06:10] Stdout logging level is INFO.


INFO:lightautoml.automl.presets.base:Stdout logging level is INFO.


[13:06:10] Task: binary



INFO:lightautoml.automl.presets.base:Task: binary



[13:06:10] Start automl preset with listed constraints:


...

[13:09:28] Automl preset training completed in 84.66 seconds



INFO:lightautoml.automl.presets.base:Automl preset training completed in 84.66 seconds



[13:09:28] Model description:
Final prediction for new objects (level 0) = 
	 0.12830 * (5 averaged models Lvl_0_Pipe_0_Mod_0_LinearL2) +
	 0.08984 * (5 averaged models Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM) +
	 0.05511 * (5 averaged models Lvl_0_Pipe_1_Mod_2_CatBoost) +
	 0.05054 * (5 averaged models Lvl_0_Pipe_1_Mod_4_XGBoost) +
	 0.67621 * (5 averaged models Lvl_0_Pipe_1_Mod_5_Tuned_XGBoost) 



INFO:lightautoml.automl.presets.base:Model description:
Final prediction for new objects (level 0) = 
	 0.12830 * (5 averaged models Lvl_0_Pipe_0_Mod_0_LinearL2) +
	 0.08984 * (5 averaged models Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM) +
	 0.05511 * (5 averaged models Lvl_0_Pipe_1_Mod_2_CatBoost) +
	 0.05054 * (5 averaged models Lvl_0_Pipe_1_Mod_4_XGBoost) +
	 0.67621 * (5 averaged models Lvl_0_Pipe_1_Mod_5_Tuned_XGBoost) 



CPU times: user 2min 20s, sys: 2.23 s, total: 2min 22s
Wall time: 1min 24s


**Сохранение модели**

In [20]:
# сохраним модель
with open(model_file,'wb') as save_model:
    joblib.dump(model,save_model)

In [21]:
# загрузим модель
with open(model_file,'rb') as save_model:
    model = joblib.load(save_model)
print(model.create_model_str_desc())

Final prediction for new objects (level 0) = 
	 0.12830 * (5 averaged models Lvl_0_Pipe_0_Mod_0_LinearL2) +
	 0.08984 * (5 averaged models Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM) +
	 0.05511 * (5 averaged models Lvl_0_Pipe_1_Mod_2_CatBoost) +
	 0.05054 * (5 averaged models Lvl_0_Pipe_1_Mod_4_XGBoost) +
	 0.67621 * (5 averaged models Lvl_0_Pipe_1_Mod_5_Tuned_XGBoost) 
Final prediction for new objects (level 0) = 
	 0.12830 * (5 averaged models Lvl_0_Pipe_0_Mod_0_LinearL2) +
	 0.08984 * (5 averaged models Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM) +
	 0.05511 * (5 averaged models Lvl_0_Pipe_1_Mod_2_CatBoost) +
	 0.05054 * (5 averaged models Lvl_0_Pipe_1_Mod_4_XGBoost) +
	 0.67621 * (5 averaged models Lvl_0_Pipe_1_Mod_5_Tuned_XGBoost) 


**Метрики на train**

In [22]:
thr = 0.5
train_metrics = model_metric_perfomance(model, train_full if full_train_flag else train, cv_iter=None,**{'threshold':thr})
print('Метрики на обучении:',train_metrics)
target = model.reader.target
train_pred = model.predict(train_full if full_train_flag else train)
train_pred_round = (train_pred.data.reshape(-1) >= thr).astype(int)
print(f"Целевая метрика {config_model['task']['metric']} на train для {target}:",
      model.task.metric_func(train_full[target] if full_train_flag else train[target],train_pred.data.reshape(-1)))
print(f"Целевая метрика {config_model['task']['metric']} округленная на train для {target}:",
      model.task.metric_func(train_full[target] if full_train_flag else train[target],train_pred_round))
print('Количество используемых фичей:', len(model.reader.used_features))

Метрики на обучении: {'N': 15987, 'accuracy': 0.8484393569775442, 'precision': 0.6269791006966434, 'recall': 0.6139534883720931, 'f1': 0.6203979320068934, 'roc-auc original': 0.8130308121615892, 'roc-auc_hack': 0.7608241035341111, 'pr-auc': 0.5361055685524544, 'target_label_count_0': 12762, 'target_label_count_1': 3225, 'pred_label_count_0': 12829, 'pred_label_count_1': 3158, 'auc': 0.8130308121615892}
Целевая метрика auc на train для target: 0.8130308121615892
Целевая метрика auc округленная на train для target: 0.7608241035341111
Количество используемых фичей: 1
Метрики на обучении: {'N': 15987, 'accuracy': 0.8484393569775442, 'precision': 0.6269791006966434, 'recall': 0.6139534883720931, 'f1': 0.6203979320068934, 'roc-auc original': 0.8130308121615892, 'roc-auc_hack': 0.7608241035341111, 'pr-auc': 0.5361055685524544, 'target_label_count_0': 12762, 'target_label_count_1': 3225, 'pred_label_count_0': 12829, 'pred_label_count_1': 3158, 'auc': 0.8130308121615892}
Целевая метрика auc на 